# API Decoder

In [ ]:
from pathlib import Path
import pandas as pd
import gc
import joblib
from tqdm import tqdm
# ------ Custom Imports ------
from sarpyx.processor.core.decode import S1L0Decoder
from sarpyx.utils.io import find_dat_file
# ------ Configure pandas display
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
# ------ Setup paths ------
cwd = Path.cwd().parent
data_dir = cwd / 'extracted_data'
output_dir = cwd / 'decoded_data'
# ------ Functions ------
def decode_s1_l0(input_file, output_dir):
    decoder = S1L0Decoder()
    decoder.decode_file(input_file, output_dir, save_to_zarr=True, headers_only=False)
    del decoder
    # Garbage collection
    gc.collect()


def retrieve_input_files(safe_folders, verbose=False):
    """Retrieve input files from SAFE folders."""
    pols = ['vh', 'vv', 'hh', 'hv']
    input_files = []
    folders_map = {x: [] for x in safe_folders}
    for folder in safe_folders:
        for pol in pols:
            try:
                input_file = find_dat_file(folder, pol)
                input_files.append(input_file)
                folders_map[folder].append(input_file)
                if verbose:
                    print(f'📁 Found {input_file.name} in {folder.name}')
            except FileNotFoundError:
                continue
        else:
            if verbose:
                print(f'📁 No .dat file found in {folder.name}, checking next folder...')
            pass
    return input_files, folders_map
            

# Find first SAFE folder and .dat file
safe_folders = [f for f in data_dir.rglob('*.SAFE') if f.is_dir()]
input_files, folders_map = retrieve_input_files(safe_folders)

In [2]:
# dump input_files and folders_map to joblib files
joblib.dump(input_files, output_dir / 'info_input_files.joblib')
joblib.dump(folders_map, output_dir / 'info_folders_map.joblib')

['/Data_large/marine/PythonProjects/SAR/sarpyx/decoded_data/info_folders_map.joblib']

In [ ]:
# Process a single file
single_file = input_files[0]  # Replace 0 with the desired index
if single_file.is_file():
    print(f'🔍 Processing {single_file.name}...')
    decode_s1_l0(single_file, output_dir)
else:
    print(f'❌ {single_file.name} is not a valid file.')
